In [1]:
import numpy as np 
import pandas as pd 
import random
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K
from shutil import copyfile, rmtree

In [2]:
!ls

README.md                     Untitled.ipynb
TB_Chest_Radiography_Database


In [3]:
!ls TB_Chest_Radiography_Database

Data_split                 README.md.txt
Normal                     Tuberculosis
Normal.metadata.xlsx       Tuberculosis.metadata.xlsx


In [4]:
!ls TB_Chest_Radiography_Database/Data_split

Test       Train      Validation


In [5]:
# Instantiating a generator object and normalizing the RGB values
traingen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
testgen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
valgen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_data = traingen.flow_from_directory(
    directory='TB_Chest_Radiography_Database/Data_split/Train',
    target_size=(512, 512),
    class_mode='binary',
    batch_size=20,
    seed=42
)

test_data = testgen.flow_from_directory(
    directory='TB_Chest_Radiography_Database/Data_split/Test',
    target_size=(512, 512),
    class_mode='binary',
    batch_size=20,
    seed=42
)

val_data = valgen.flow_from_directory(
    directory='TB_Chest_Radiography_Database/Data_split/Validation',
    target_size=(512, 512),
    class_mode='binary',
    batch_size=20,
    seed=42
)

Found 2940 images belonging to 2 classes.
Found 840 images belonging to 2 classes.
Found 420 images belonging to 2 classes.


In [6]:
input_shape = (512,512,3)

In [7]:
# Model building in a single list
baseline = keras.Sequential(
    [
        keras.Input(shape=input_shape), # Don't always need this input separately
        layers.Flatten(), # need to flatten our images to be one long array
        layers.Dense(64, activation="tanh"),
        layers.Dense(1, activation="sigmoid"), #sigmoid for binary
    ])

baseline.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 786432)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                50331712  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 50,331,777
Trainable params: 50,331,777
Non-trainable params: 0
_________________________________________________________________


In [8]:
batch_size = 20
epochs = 10

# Compiling our model
baseline.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["acc"])
#loss different for binary

In [9]:
baseline_results = baseline.fit_generator(train_data,
                              steps_per_epoch=20,
                              epochs=10,
                              validation_data=test_data)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
20/20 [==============================] - 15s 744ms/step - loss: 1.3646 - acc: 0.8050 - val_loss: 1.1528 - val_acc: 0.8333
Epoch 2/10
20/20 [==============================] - 15s 733ms/step - loss: 0.9670 - acc: 0.8375 - val_loss: 0.8148 - val_acc: 0.8333
Epoch 3/10
20/20 [==============================] - 14s 720ms/step - loss: 0.6299 - acc: 0.8475 - val_loss: 0.5604 - val_acc: 0.8333
Epoch 4/10
20/20 [==============================] - 14s 700ms/step - loss: 0.4207 - acc: 0.8675 - val_loss: 0.4712 - val_acc: 0.8333
Epoch 5/10
20/20 [==============================] - 15s 725ms/step - loss: 0.4863 - acc: 0.8175 - val_loss: 0.4506 - val_acc: 0.8333
Epoch 6/10
20/20 [==============================] - 14s 682ms/step - loss: 0.4540 - acc: 0.8325 - val_loss: 0.4510 - val_acc: 0.8333
Epoch 7/10
20/20 [==============================] - 13s 665ms/step - loss: 0.4450 - acc: 0.8375 - val_loss: 0.4507 - val_acc: 